# Lokales Embedding benutzen

Dieses Notebook führt dich durch folgende Schritte:

- Installation der benötigten Pakete.
- Benutzen des Sitemap Loaders um die deepshore.de nach Wissensbeiträgen zu durchkämmen.
- Die Dokumente an ein *lokales* Embeddings Modell schicken und in Vectoren verwandeln und im Index speichern.
- Eine Frage an das *lokale* Embeddings Modell schicken und die neu gewonnenen Vectoren zusammen mit den indexierten Vectoren verorten und eine Antwort bekommen


Du benötigst:

- python 3.10
- Jupyter Notebook Server
- Einen OpenAI API Token

# Installation

In [ ]:
%pip install langchain=="0.0.222"
%pip install "langchain[llms]"=="0.0.222"
%pip install gpt4all=="0.3.5"
%pip install chromadb=="0.3.26"
%pip install llama_index=="0.6.38"
%pip install llama_hub=="0.0.5"
%CMAKE_ARGS="-DLLAMA_METAL=1" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall

In [ ]:
import os
import sys
print(os.path.dirname(sys.executable))
print(os.getcwd())

!python --version

## Dokumente laden

... indem man den Sitemap Loaders benutzt, um die Webseite deepshore.de nach Wissensbeiträgen zu durchkämmen

In [ ]:
from llama_hub.web.sitemap.base import SitemapReader

import nest_asyncio
nest_asyncio.apply()

loader = SitemapReader(html_to_text=True)
documents = loader.load_data(sitemap_url='https://deepshore.de/sitemap.xml', filter='https://deepshore.de/knowledge')

print(len(documents))

# Model laden

Folgendes modell wurde hier heruntergeladen: [hugging face](https://huggingface.co/TheBloke/open-llama-7b-open-instruct-GGML)

In [ ]:
import os
from langchain.llms import LlamaCpp
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager

callbacks = [StreamingStdOutCallbackHandler()]
callback_manager = CallbackManager(callbacks)

model_name = f"{os.getcwd()}/models/open-llama-7B-open-instruct.ggmlv3.q6_K.bin"

llm = LlamaCpp(model_path=model_name, callback_manager=callback_manager, verbose=True, use_mlock=False, n_gpu_layers=10000)

## Die Dokumente an das lokale Embeddings Modell schicken

...und in Vektoren verwandeln und im Index speichern 

In [ ]:
# import
from langchain.embeddings.llamacpp import LlamaCppEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from llama_index.schema import Document

langchain_documents = []
for d in documents:
    langchain_documents.append(d.to_langchain_format())

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(langchain_documents)

# create the open-source embedding function
embedding_function = LlamaCppEmbeddings(model_path=model_name, use_mlock=True, n_gpu_layers=1000, n_threads=8, n_ctx=2048, n_batch=500)

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)


## Eine Frage an das Modell schicken

... und eine Antwort im Kontext der Daten bekommen